In [1]:
###
# inside acquire.py script:
from env import uname, pwd, host
import env
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
###
######### USE THIS FOR THE zillow DATASET!!!!

def get_df():
    '''
    This function reads the telco data from the Codeup db into a df.
    '''
    
    filename = "zillow.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename,index_col=False)
    else:
        sql_query = """
                SELECT  bedroomcnt as beds,
                    bathroomcnt as baths,
                    calculatedfinishedsquarefeet as sqft,
                    taxvaluedollarcnt as taxable_value,
                    yearbuilt as built,
                    taxamount as tax,
                    fips,
                    propertylandusetypeid
                FROM properties_2017
                WHERE propertylandusetypeid = 261
                """
    
        # Read in DataFrame from Codeup db.
        df = pd.read_sql(sql_query, env.get_db_url('zillow'))
        df.to_csv(filename,index=False)
        return df


#----------------------------------------------------

In [3]:
df = get_df()
df.head()

,beds,baths,sqft,taxable_value,built,tax,fips,propertylandusetypeid
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0


In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
cols = [col for col in df.columns if col not in ['fips','built']]
cols

In [ ]:
plt.figure(figsize=(20,10))
for i, col in enumerate(cols):
    plot_number = i+1
    plt.subplot(1,len(cols),plot_number)
    plt.title(f'Dist of {col}')
    df[col].hist(bins=10)
    plt.grid(False)
    plt.ticklabel_format(useOffset=False)
plt.show()

In [ ]:
sns.boxplot(data=df)

In [ ]:
df.isna().sum()

In [ ]:
df['baths'].describe()['25%']

In [ ]:
df['baths'].quantile(0.25)

In [ ]:
# IQR
iqr = df['baths'].quantile(0.75) - df['baths'].quantile(0.25)
iqr

In [ ]:
lower_bath_fence = (df['baths'].quantile(0.25)) - (1.5*iqr)
lower_bath_fence

In [ ]:
upper_bathroom_fence = (df['baths'].quantile(0.75)) + (1.5*iqr)
upper_bathroom_fence

In [ ]:
df[(df.baths > lower_bath_fence) & (df.baths < upper_bathroom_fence)].baths.describe()

In [ ]:
col_qs = {}
for col in cols:
    col_qs[col] = q1,q3 = df[col].quantile([0.25,0.75])

In [ ]:
col_qs['beds'][0.25]

In [ ]:
def remove_outliers(df,col_list,k=1.5):
    '''
    remove outliers based on column list usinf tukey 1.5iqr
    returns a single df w/o outliers
    '''
    col_qs = {}
    for col in col_list:
        col_qs[col] = q1,q3 = df[col].quantile([0.25,0.75])
    for col in col_list:
        iqr = col_qs[col][0.75] - col_qs[col][0.25]
        lower_fence = col_qs[col][0.25] - (k*iqr)
        upper_fence = col_qs[col][0.75] + (k*iqr)
        df = df[(df[col] > lower_fence) & (df[col] < upper_fence)]
    return df

In [ ]:
smol_df = remove_outliers(df,cols)
smol_df.shape

In [ ]:
df.info()

## First, we drop the property land use id...then we can drop rows with missingness...
## and finally we can convert all floats to INT

In [4]:
###
def clean_zillow(df):
    '''
    clean_szillow will take in df and will remove propertyland..., rows with NULL values in any cell and will
    cast floats into int.
    
    args: df
    return: df (clean)
    
    '''
    df = df.drop(columns='propertylandusetypeid')
    df = df.dropna()
    df = df.astype(int)
    return df

In [5]:
df = clean_zillow(df)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [6]:
###
from sklearn.model_selection import train_test_split

In [7]:
###
def split_zillow(df):
    train_val,test = train_test_split(df,
                                     random_state=2013,
                                     train_size=0.7)
    train, validate = train_test_split(train_val,
                                      random_state=2013,
                                      train_size=0.8)
    return train, validate, test

In [9]:
train, validate, test = split_zillow(df)

In [10]:
test.shape

(642071, 7)

In [ ]:
###
def wrangle_zillow():
    return split_zillow(
            clean_zillow(get_df()))

In [ ]:
# NOT THIS
df2 = wrangle_zillow()
df2

In [ ]:
### THIIIIIIIISSSSSSS!!!!
train,validate,test = wrangle_zillow()
train

#### Simple Explore

In [ ]:
train.shape,validate.shape,test.shape

In [ ]:
for col in train:
    train[col].hist()
    plt.title(col)
    plt.show()

In [ ]:
for col in train:
    sns.boxplot(data=train, x=col)
    plt.title(col)
    plt.show()

In [ ]:
train.describe().T

In [ ]:
train.info()

In [ ]:
sns.pairplot(train,corner=True)
plt.show()